### Задачи к Лекции 4

__Исходные данные__ 

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from itertools import * 
warnings.filterwarnings('ignore')

df = pd.read_csv("../data/mlbootcamp5_train.csv", 
                 sep=";", 
                 index_col="id")
df = df.dropna(subset=['age','gender','height','weight','ap_hi','ap_lo','cholesterol','gluc','smoke','alco','active']).sort_values('id')
df_cat = df[(df["ap_hi"] >= 100) &
          (df["ap_hi"] <= 200) &
          (df["ap_lo"] >= 50) & 
          (df["ap_lo"] <= 150) &
          (df["weight"] >= 40)
         ]
# Делаем пол бинарным признаком
df_cat["gender_bin"] = df_cat["gender"].map({1: 0, 2: 1})
df_cat = df_cat.drop(["gender"],axis=1)
target_cat = df_cat["cardio"]


# Делаем one-hot кодирование
chol = pd.get_dummies(df_cat["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df_cat["gluc"], prefix="gluc")
df_one_hot = pd.concat([df_cat, chol, gluc], axis=1)

target_data = df_one_hot["cardio"]
df_one_hot.head()
df_cat.head()

,age,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,gender_bin
id,,,,,,,,,,,,
0,18393,168,62.0,110,80,1,1,0,0,1,0,1
1,20228,156,85.0,140,90,3,1,0,0,1,1,0
2,18857,165,64.0,130,70,3,1,0,0,0,1,0
3,17623,169,82.0,150,100,1,1,0,0,1,1,1
4,17474,156,56.0,100,60,1,1,0,0,0,0,0


## Классы в Python

Нередко, возникает необходимость создания объектов с каким-нибудь внутренним поведением и состоянием. Примерами таких объектов являются классификаторы sklearn, массивы numpy и много другое. Такой объект можно объявить с помощью ключевого слова **class**

```python
class SomeObject:
    def __init__(self, depth):
        self.a = depth
        self.target = None
        
    def fit(self, data, target):
        self.target = data
        # magic
        return 
    
    def predict(self, data):
        return self.target    
```

После этого в коде можно будет создать экземпляр данного класса
```python
a = SomeObject(depth=5)
a.fit(data, target)
a.predict(data)
```

## Задачи

**1. В sklearn на данный момент отсутствует функционал для построения деревьев решений из категориальных данных, поэтому его нужно сделать самостоятельно и проверить его работу. Что нужно сделать:**

* __создать классификатор используя только pandas, numpy и scipy. Необходимо его сделать самому, используя исключительно только numpy, pandas и scipy (запрещено использовать sklearn и прочие библиотеки). Напоминаю, что для категориальных данных операция < или > не имеют смысла (использовать только != и ==). Гиперпараметром данного классификатора должна быть максимальная глубина дерева.__
* __Проверить работу данного классификатора на наборе ("gender", "cholesterol", "gluc").__
* __С помощью кросс-валидации найти оптимальную глубину этого дерева. Для вашего классификатора GridSearchCV не подойдет, придется это сделать также самостоятельно.__
* __Нарисовать полученное дерево (я должен понять, как и откуда вы его нарисовали).__

Алгоритм работы классификатора:
 1. Перебираем все возможные признаки и смотрим либо неопределенность Джини, либо прирост информации. Это даст критерий разбиения в виде "признак == значение"
 2. Если выборка полученная при разбиении состоит из объектов одного класса (соответсвует нулевой энтропии), то данный лист просто возвращает значение этого класса.
 3. В противном случае, образуется новый узел и для него начинаем с пункта 1.
 4. Если достигли максимальной глубины, то вместа узла создаем лист, который возвращает самое вероятное значение.
 
__Замечание:__ в этой задаче не нужно использовать onehot-кодирование.

In [2]:
# A lot of code here

**Комментарии:** Ваши комментарии здесь.

**2. В этой задаче и далее можно использовать sklearn. Выше, данные были приведены либо к бинарным признакам, либо к количественным. Это позволяет воспользоваться классификатором DecisionTreeClassifier. Нужно с помощью кросс-валидации найти оптимальный набор признаков. Показать и объяснить, как данный набор был получен.**

In [3]:
%%time
df_cat_no_cardio = df_cat.drop(['cardio'],axis=1)
tree = DecisionTreeClassifier(max_depth=10, random_state=13)
df_cross = pd.DataFrame(columns = ['df_cat_names','cross_val_accuracy'])
field_list = [i for i in df_cat_no_cardio.columns]
for i in range(2,df_cat_no_cardio.shape[1]):
    #if i != 2:
        #break
    for j in combinations(field_list,i):
        cv = cross_val_score(tree, 
                    df_cat[list(j)],
                    target_cat,
                    n_jobs=-1,
                    scoring='accuracy',
                    cv=5)
        df_cross.loc[len(df_cross.index)] = [list(j), cv.mean()]     
df_cross.sort_values('cross_val_accuracy',ascending = False).head(1)

Wall time: 7min 54s


,df_cat_names,cross_val_accuracy
1217,"[age, ap_hi, ap_lo, cholesterol, active, gende...",0.726318


**Комментарии:** Ваши комментарии здесь.

**3. Для набора признаков полученных в задаче 3, с помощью кросс-валидации найти оптимальный набор гиперпараметров.**

In [27]:
%%time
#max_features
df_max_features = df_cross.sort_values('cross_val_accuracy',ascending = False).head(1).values[0][0]#TUPA_KEK
print(df_max_features)
train, test, target_train, target_test = train_test_split(
    df_cat[df_max_features], target_cat, 
    test_size=0.3
)

#Finding the optimal values of "max_depth" and "min_samples_leaf" by GridSearchCV
params = {
    'max_depth': np.arange(2, 20, 1),
    'min_samples_leaf': np.arange(2, 20, 2),
    'max_features' : np.arange(1, len(df_max_features), 1)
}
tree_hyper = DecisionTreeClassifier(random_state=13)
grid = GridSearchCV(
    tree, params, scoring='accuracy',n_jobs=-1,
    cv=5,
)
grid.fit(train, target_train);
print(grid.best_params_,grid.best_score_)

['age', 'ap_hi', 'ap_lo', 'cholesterol', 'active', 'gender_bin']
{'max_depth': 7, 'max_features': 5, 'min_samples_leaf': 16} 0.7299145006246287
Wall time: 20.4 s


**Комментарии:** Ваши комментарии здесь.

**4. Сравнить с помощью кросс-валидации классификатор "наивный байес" и "решающее дерево" на количественных признаках. Показать, какой из них лучше.**

In [29]:
from sklearn.naive_bayes import GaussianNB

print("Точность(DecisionTree_best) = %f" % grid.best_score_)

train_4 = df_cat[df_max_features]
target_4 = target_cat

gnb = GaussianNB()
model = gnb.fit(train_4, target_4)
gprob = model.predict_proba(train_4)
gpred = model.predict(train_4)
acc = accuracy_score(target_4, gpred)
cv_gbn = cross_val_score(model, 
                    train_4,
                    target_4,
                    n_jobs=-1,
                    scoring='accuracy',
                    cv=5)
print("Точность(gbn) = %f" % acc)
print (cv_gbn, cv_gbn.mean())

#print(df_cat.shape[0],
     #(target_5 == predict_num).sum() / df_cat.shape[0])


Точность(DecisionTree_best) = 0.729915
Точность(gbn) = 0.717473
[0.71740741 0.72014815 0.71703704 0.7202015  0.7125713 ] 0.7174730781155256


**Комментарии:** Ваши комментарии здесь.